In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
from sklearn.svm import SVC
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 30000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [3]:
df= pd.read_csv('data/df_pre.csv')

In [5]:
df.tail(5)

,Unnamed: 0,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Color_pre,DayuponOutcome,AboutAge,BreedMix,Breed_pre,Year,Month,YearMonth,Season,Quarter,Group,Size
26724,26724,18631,0,14983,4,Partner,0,1,2,640,111,2,30,2,1,80,2,4,19,1,1,8,5
26725,26725,26055,0,22209,4,SCRP,0,3,24,640,104,2,90,2,1,80,3,0,27,3,0,8,5
26726,26726,16731,1,13463,0,NaN,1,2,30,1022,353,22,1460,20,1,147,2,2,17,1,0,12,9
26727,26727,7450,0,5074,4,Partner,0,1,29,640,5,0,28,2,1,80,1,3,6,1,1,8,5
26728,26728,20751,0,16611,4,SCRP,0,1,5,640,111,2,365,2,1,80,2,6,21,2,2,8,5


In [6]:
del df['Unnamed: 0']

In [7]:
df.tail()

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Color_pre,DayuponOutcome,AboutAge,BreedMix,Breed_pre,Year,Month,YearMonth,Season,Quarter,Group,Size
26724,18631,0,14983,4,Partner,0,1,2,640,111,2,30,2,1,80,2,4,19,1,1,8,5
26725,26055,0,22209,4,SCRP,0,3,24,640,104,2,90,2,1,80,3,0,27,3,0,8,5
26726,16731,1,13463,0,NaN,1,2,30,1022,353,22,1460,20,1,147,2,2,17,1,0,12,9
26727,7450,0,5074,4,Partner,0,1,29,640,5,0,28,2,1,80,1,3,6,1,1,8,5
26728,20751,0,16611,4,SCRP,0,1,5,640,111,2,365,2,1,80,2,6,21,2,2,8,5


In [8]:
df_y = pd.DataFrame(data = df , columns = ['OutcomeType'])

In [9]:
columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_X = pd.DataFrame(data = df , columns = columns)

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.1, random_state=0 , shuffle = True)

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24056, 8)
(24056, 1)
(2673, 8)
(2673, 1)


In [11]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [12]:
%%time
rbfsvc = SVC(kernel="rbf",probability=True).fit(X_train_std, y_train)

CPU times: user 3min 24s, sys: 2.09 s, total: 3min 26s
Wall time: 3min 27s


In [13]:
rbf_probs = rbfsvc.predict_proba(X_test_std)

In [14]:
print('train score: {}'.format(rbfsvc.score(X_train_std, y_train)))
print('test score: {}'.format(rbfsvc.score(X_test_std, y_test)))
print('log loss: {}'.format(log_loss(y_test, rbf_probs)))
print()

train score: 0.64865314266711
test score: 0.6442199775533108
log loss: 0.9092947840430579



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(5, shuffle=True)

pipe_svc = Pipeline([('scl', StandardScaler()), ('clf', SVC(random_state=0))])

param_range = [0.1, 1.0, 10.0]
param_grid = [{'clf__C': param_range, 'clf__gamma': param_range, 'clf__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=cv)
%time gs = gs.fit(X_train_std, y_train)